In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import dotenv_values, find_dotenv
import re

In [2]:
# this looks for your configuration file and then reads it as a dictionary
config = dotenv_values(find_dotenv())

# set path using the dictionary key for which one you want
concordancedata = os.path.abspath(config["DATA_CONCORDANCE"]) + '\\'

In [4]:
# import data

# https://econweb.ucsd.edu/~jrauch/rauch_classification.html
rauch = pd.read_csv(concordancedata + 'Rauch_classification_revised.csv')
# https://unstats.un.org/unsd/trade/hs2_2sitc3.htm
sitc_hs6 = pd.read_csv(concordancedata + 'HS6 to STIC rev3 concordance - unstats.csv')[['HS02', 'SITC, R3']]
# rename columns for merging later
sitc_hs6.rename(columns={'HS02': 'hs6', 'SITC, R3': 'sitc4'}, inplace=True)
# switch to string datatype for easier data cleaning
sitc_hs6['hs6']= sitc_hs6['hs6'].astype(str)
sitc_hs6['sitc4']= sitc_hs6['sitc4'].astype(str)
rauch['sitc4'] = rauch['sitc4'].astype(str)

# https://econweb.ucsd.edu/~jrauch/rauchclass/SITC_Rev_3_english_structure.txt
with open(concordancedata + 'SITC_Rev_3_english_structure.txt', 'r', errors='ignore') as file:
    lines = file.readlines()

# refine a regex pattern for splitting the lines
pattern = re.compile(r'(\S+)\s+(.+)')

# df columns
code_list = []
description_list = []

# iterate through lines and extract code and description
for line in lines:
    match = pattern.match(line)
    if match:
        code_list.append(match.group(1))
        description_list.append(match.group(2))

sitc_desc = pd.DataFrame({'sitc4': code_list[1:], 'desc': description_list[1:]})

# keep sitc 4 digit long codes
sitc_desc['sitc4'] = sitc_desc['sitc4'].str.replace('.', '', regex=True)
sitc_desc = sitc_desc[sitc_desc['sitc4'].str.len() == 4]

In [15]:
# restrict to our product categories
sitc_hs6 = sitc_hs6.loc[sitc_hs6['hs6'].str.startswith('85') | sitc_hs6['hs6'].str.startswith('61') | sitc_hs6['hs6'].str.startswith('62')]

# fix formatting for sitc codes in order to merge with rauch codes file
sitc_hs6['sitc4'] = sitc_hs6['sitc4'].str.replace('.', '', regex=True)
sitc_hs6['sitc4'] = sitc_hs6['sitc4'].apply(lambda x: x[:-1] if len(x) == 5 else x)

In [16]:
# do the merge
concordance = pd.merge(left=sitc_hs6, right=rauch, how='left', on='sitc4')
concordance = pd.merge(left=concordance, right=sitc_desc, how='left', on='sitc4')

In [17]:
# unmatched products
concordance[concordance['lib'].isna()]

,hs6,sitc4,con,lib,desc
256,8503.0,7169,NaN,NaN,"Parts, n.e.s., suitable for use solely or prin..."
309,8513.9,8138,NaN,NaN,Parts of the portable electric lamps of headin...
375,8523.2,8985,NaN,NaN,Other prepared unrecorded media for sound reco...
376,8523.3,8985,NaN,NaN,Other prepared unrecorded media for sound reco...
377,8523.9,8985,NaN,NaN,Other prepared unrecorded media for sound reco...
378,8524.1,8987,NaN,NaN,Records and other recorded media (excluding ma...
379,8524.31,8987,NaN,NaN,Records and other recorded media (excluding ma...
380,8524.32,8987,NaN,NaN,Records and other recorded media (excluding ma...
381,8524.39,8987,NaN,NaN,Records and other recorded media (excluding ma...
386,8524.6,8987,NaN,NaN,Records and other recorded media (excluding ma...


In [18]:
# non-differentiated products (by "liberal" classification)
concordance[(concordance['lib'] != 'n') & (concordance['lib'].notnull())]

,hs6,sitc4,con,lib,desc
268,8505.11,7788,n,r,"Electrical machinery and equipment, n.e.s."
269,8505.19,7788,n,r,"Electrical machinery and equipment, n.e.s."
270,8505.2,7788,n,r,"Electrical machinery and equipment, n.e.s."
271,8505.3,7788,n,r,"Electrical machinery and equipment, n.e.s."
272,8505.9,7788,n,r,"Electrical machinery and equipment, n.e.s."
273,8506.1,7781,n,r,"Batteries and electric accumulators, and parts..."
274,8506.3,7781,n,r,"Batteries and electric accumulators, and parts..."
275,8506.4,7781,n,r,"Batteries and electric accumulators, and parts..."
276,8506.5,7781,n,r,"Batteries and electric accumulators, and parts..."
277,8506.6,7781,n,r,"Batteries and electric accumulators, and parts..."


In [28]:
# dictionary of unclassified sitc codes
unmatched = concordance[concordance['lib'].isna()].drop_duplicates(subset=['sitc4', 'con', 'lib', 'desc'])
sitc_unmatched = dict(zip(unmatched['sitc4'], unmatched['desc']))

In [29]:
unmatched

,hs6,sitc4,con,lib,desc
256,8503.0,7169,NaN,NaN,"Parts, n.e.s., suitable for use solely or prin..."
309,8513.9,8138,NaN,NaN,Parts of the portable electric lamps of headin...
375,8523.2,8985,NaN,NaN,Other prepared unrecorded media for sound reco...
378,8524.1,8987,NaN,NaN,Records and other recorded media (excluding ma...
428,8533.1,7723,NaN,NaN,Electrical resistors (including rheostats and ...
436,8535.1,7724,NaN,NaN,Electrical apparatus for switching or protecti...
451,8537.1,7726,NaN,NaN,"Boards, panels (including numerical control pa..."
453,8538.1,7728,NaN,NaN,Parts suitable for use solely or principally w...
492,8543.11,7787,NaN,NaN,"Electrical machines and apparatus, having indi..."


In [30]:
# there are 9 SITC codes that don't have a Rauch classification (let ChatGPT fill in the matching?)
sitc_unmatched

{'7169': 'Parts, n.e.s., suitable for use solely or principally with the machines falling within group 716',
 '8138': 'Parts of the portable electric lamps of heading 813.12 (excluding storage batteries)',
 '8985': 'Other prepared unrecorded media for sound recording or similar recording of other phenomena (excluding products of group 883)',
 '8987': 'Records and other recorded media (excluding magnetic tapes) for sound or other similarly recorded phenomena (including matrices and masters for the production of records, but excluding products of group 883)',
 '7723': 'Electrical resistors (including rheostats and potentiometers), other than heating resistors; parts thereof',
 '7724': 'Electrical apparatus for switching or protecting electrical circuits or for making connections to or in electrical circuits (e.g., switches, fuses, lightning arresters, voltage limiters, surge suppressors, plugs, junction boxes), for a voltage exceeding 1',
 '7726': 'Boards, panels (including numerical con

In [ ]:
# dictionary of products with differing "liberal" classification than conservative
conflicting_matches = concordance[(concordance['lib'] != concordance['con']) & (concordance['lib'].notnull())].drop_duplicates(subset=['sitc4', 'con', 'lib', 'desc'])
sitc_conflicting_matches_desc = list(conflicting_matches['desc'])

In [8]:
# save concordance file
concordance.to_csv(concordancedata + 'rauch_hs6_sitc.csv')